In [27]:
import torch
import torchvision
import numpy as np
from pathlib import Path
from glob import glob
import cv2 
import matplotlib.pyplot as plt 
from PIL import Image
import scipy.io
from tqdm import trange, tqdm
from random import shuffle
import pickle

from src.alexnet import model as alexnet_model, preprocess as alexnet_preprocess
from data.labels import labels

from src.simba import SimBA_Attack

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Device used: {device}")

Device used: cuda:0


In [28]:
labels_path = Path("./results/used_labels.txt")
image_names_path = Path("./results/used_images.txt")
images_subset_path = Path("./results/subset_used/")

In [29]:
def get_names_labels():
    with open(labels_path, "r") as f:
        lines = f.readlines()
    labels = [line.rstrip() for line in lines]
    
    with open(image_names_path, "r") as f:
        lines = f.readlines()
    names = [line.rstrip() for line in lines]
    return labels, names
        
LABELS, NAMES = get_names_labels()
PATHS = ["./results/subset_used/" + p for p in NAMES]

In [30]:
def get_alexnet_img(path):
    img = Image.open(path)
    return alexnet_preprocess(img)

def find_correctly_classified(model, paths, labels):
    """
        Shuffle paths / labels together so we get a different set of inputs
        each time.
        
        Ref:
        https://www.geeksforgeeks.org/python-shuffle-two-lists-with-same-order/
    """
        
    model.eval()
        
    correct = []
    for i in trange(len(paths)):
        this_path = paths[i]
        
        try:
            this_image = get_alexnet_img(this_path)
            
            with torch.no_grad():
                logits = model(this_image.view(-1,3,224,224))
                
            preds = torch.nn.functional.softmax(logits, dim=1)
        except:
            continue
            
        y_hat = int(torch.argmax(preds))
        
        """
            Found that inputs that model was very confident on required
            many adversarial iterations, so picked those that it is reason-
            ably confident on but is somewhat close to the decision boundary."""
        if y_hat == int(labels[i]):
            correct += [[this_path, y_hat]]
            
    return correct

In [31]:
n_correct = find_correctly_classified(alexnet_model, PATHS, LABELS)

100%|█████████████████████████████████████████| 987/987 [00:14<00:00, 70.28it/s]


In [32]:
print(len(n_correct))

987


In [33]:
# grab datastructure w/ perturbations from pickle
with open("./results/sorted_outputs.p", "rb") as f:
    out_vecs_read = pickle.load(f)
    
print(len(out_vecs_read))

987
